In [1]:
import os
import numpy as np
import torch
import yaml
from ipywidgets import interact
import ipywidgets as widgets
from matplotlib.pylab import plt
from tqdm.notebook import tqdm
import imageio
import pandas as pd

# from src.engine import load_model, val_tfs
from src.dataset import val_tfs
from src.evaluate import iou, filter_pixels
from src.dataset import ChipsDataset

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12);

with open('./src/config.yaml') as f:
    config = yaml.safe_load(f)
config['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'

/home/quantum/miniconda3/envs/soil_erosion/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [14]:
import segmentation_models_pytorch as smp
import albumentations as A
# import torchvision.models as models
from src.high_resolution_net.lib.models.hrnet import hrnet48, hrnet32, hrnet18

def make_unet(backbone='resnet34', weigths='imagenet',
              num_classes=1, activation='sigmoid', channels=3):

    model = smp.Unet(
        encoder_name=backbone,
        encoder_weights=weigths,
        classes=num_classes,
        activation=activation,
        in_channels=channels)

    return model

def make_unet_plusplus(backbone='resnet50', weigths='imagenet',
                   num_classes=1,
                   activation='sigmoid',channels=3):

    model = smp.UnetPlusPlus(
        encoder_name=backbone,
        encoder_weights=weigths,
        classes=num_classes,
        activation=activation,
        in_channels=channels)

    return model

def make_hrnet_18(channels=3):
    model = hrnet18(pretrained=False, progress=True, channels=channels)
    return model

def load_model(model_name, path=None, device='cpu', channels=3, **kwargs):

    name = model_name.split('_')[0]
#     model = make_unet(channels=channels, **kwargs)
    model = make_unet_plusplus(channels=channels, **kwargs)
#     model = make_deeplab()
    model = make_hrnet_18()

    if path is not None:
        model.load_state_dict(torch.load(path,
            map_location=config['device']))

    return model.to(device)

# model = load_model("unet", path="models/deeplab_none_40_nir_tci_multitemporal.pth", channels=3)
model = load_model("hrnet", "models/highresolutionnet18_none_40_nir_tci_multitemporal.pth")
# model = load_model("unetplusplus", path="models/UnetPlusPlus_none_40_nir_tci_multitemporal.pth", channels=3)

In [15]:
sum([p.numel() for p in model.parameters()])

9601937

In [16]:
if config['device'] == 'cuda':
    torch.cuda.empty_cache()
    
val_tfs = A.Compose([
    A.pytorch.transforms.ToTensorV2(p=1.0)])
    
dataset = ChipsDataset(transforms=val_tfs)
dataset.transforms = val_tfs

In [17]:
def visualize_results(dataset):
    def visualize_image(num):
        img, mask, _ = dataset[num]
        pred = model(img.unsqueeze(0))

        return img, mask.numpy(), pred.detach().numpy()

    def f_fabric(num):
        def f(x):
            img, mask, pred = visualize_image(x)
            fig, axes = plt.subplots(1, 3, figsize = (15, 12))

            axes[0].set_title('Image')
            axes[0].imshow(img.permute(1,2,0).numpy())
            axes[0].axis("off")

            axes[1].set_title('Prediction')
            axes[1].imshow(pred[0][0], cmap='gray')
            axes[1].axis("off")
            
            axes[2].set_title('GT mask')
            axes[2].imshow(mask[0], cmap='gray')
            axes[2].axis("off")

            plt.tight_layout()

        return f
    
    plt.rcParams.update({'font.size': 14})
    f = f_fabric(0)
    interact(f, x=widgets.IntSlider(min=0, max=len(dataset)-1, step=1, value=0, continuous_update=False))

In [18]:
visualize_results(dataset)

interactive(children=(IntSlider(value=0, continuous_update=False, description='x', max=330), Output()), _dom_c…

In [7]:
options = [round(x*0.05, 2) for x in range(1, 19)]

def find_thresh(model, optimize=iou, options=options):
    best_iou = (None, 0)
    pbar = tqdm(options)
    
    for val in pbar:
        
        conf = config.copy()
        config['threshold'] = val
        valid_ious = get_ious(model, config, 'valid_v3', bands=['tci', 'nir'])
        if np.mean(valid_ious) > best_iou[1]:
            best_iou = (val, np.mean(valid_ious))
        pbar.set_postfix({'value': val, 'iou': best_iou})
            
    return best_iou

In [8]:
thresh, iou_ = find_thresh(model)
config['threshold'] = thresh
with open('./code/config.yaml', 'w') as f:
    f.write(yaml.dump(config))

  0%|          | 0/18 [00:00<?, ?it/s]

NameError: name 'get_ious' is not defined

In [ ]:
thresh, iou_

### Model Predictions test

In [ ]:
imgs, extra_imgs, masks, filters, preds, merged, test_ious = get_gt_n_pred_imgs(
    model, config, 'test_v3', False, bands=['TCI', 'NIR'])
visualize_results(imgs, extra_imgs, masks, filters, preds, merged, test_ious)
test_ious = get_ious(model, config, 'test_v3', bands=['TCI', 'NIR'])

### Model train prediction

In [ ]:
imgs, extra_imgs, masks, filters, preds, merged, train_ious = get_gt_n_pred_imgs(
    model, config, 'train_v3', False, bands=['TCI', 'NIR'])
visualize_results(imgs, extra_imgs, masks, filters, preds, merged, train_ious)
train_ious = get_ious(model, config, 'train_v3', bands=['TCI', 'NIR'])

### Model valid predictions

In [ ]:
imgs, extra_imgs, masks, filters, preds, merged, valid_ious = get_gt_n_pred_imgs(
    model, config, 'valid_v3', False, bands=['TCI', 'NIR'])
visualize_results(imgs, extra_imgs, masks, filters, preds, merged, valid_ious)
valid_ious = get_ious(model, config, 'valid_v3', bands=['TCI', 'NIR'])

In [ ]:
results = pd.DataFrame()
results = results.append({
    'train_iou': np.mean(train_ious),
    'valid_iou': np.mean(valid_ious),
    'test_iou': np.mean(test_ious),
}, ignore_index=True)
results

In [ ]:
os.path.split(model_path)[-1].split('.')[0]+'.csv'

In [ ]:
results.to_csv(os.path.split(model_path)[-1].split('.')[0]+'.csv', index=False)